In [21]:
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Letra')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install opencv-python
!pip install torchsummary
!pip install nes-py
!pip install --force-reinstall gym-super-mario-bros==7.4.0  > /dev/null 2>&1
!pip install --force-reinstall gym==0.15.3 > /dev/null 2>&1
!pip install pillow==8.4.0
!pip install torchvision==0.11.1
!pip install pyglet==1.2.4
!pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
import os
import torch
import gym
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import mario_utils

import warnings
warnings.filterwarnings("ignore")

# Notebook de Prueba de algoritmos DQN y DDQN

In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Running on {DEVICE}")

Running on cuda:0


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class DQN_Linear_Model(nn.Module):
    def __init__(self,  n_features, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(in_features=n_features, out_features=50)
        #self.fc2 = nn.Linear(512, 256)
        self.output = nn.Linear(in_features=50, out_features= n_actions)

    def forward(self, env_input):
        result = env_input.reshape((-1, self.fc1.in_features))
        result = F.relu(self.fc1(result))
        #result = F.relu(self.fc2(result))
        return self.output(result)


In [6]:
from replay_memory import ReplayMemory, Transition

In [7]:
ENV_NAME = 'MountainCar-v0'
env = gym.make(ENV_NAME)
n_features = env.observation_space.shape[0]
n_actions = env.action_space.n

In [8]:
print(env.observation_space.low)
print(env.observation_space.high)

[-1.2  -0.07]
[0.6  0.07]


In [9]:
import numpy as np


def discretization(obs):
    # observation_space low y high nos permiten conocer los límites de nuestro ambiente para los valores de Posicion y Velocidad.
    env_low = np.array([-1.2, -0.07])
    env_high = np.array([0.6, 0.07])

    env_den = (env_high - env_low) / 40
    pos_den = env_den[0]
    vel_den = env_den[1]

    pos_low = env_low[0]
    vel_low = env_low[1]

    pos_scaled = int((obs[0] - pos_low) / pos_den)
    vel_scaled = int((obs[1] - vel_low) / vel_den)

    return pos_scaled, vel_scaled

In [10]:
def process_state(observation):
    # Transforme la observacion en un tensor de floats.
    observation = discretization(observation)
    observation = torch.tensor(observation).float()
    return observation

In [11]:
from dqn_agent import DQNAgent
# Cada vez que hacemos un experimento reseteamos la semilla para tener reproducibilidad
env.seed(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True


### Entrenamiento DQN

In [12]:
#Hiperparámetros de entrenamiento del agente DQN
TOTAL_STEPS =10000000
EPISODES = 500
STEPS = 10000000

EPSILON_INI = 1
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.99998599985
EPSILON_TIME = 2000
EPISODE_BLOCK = 110
SAVE_EVERY = 600

BATCH_SIZE = 64
BUFFER_SIZE = 10000

GAMMA = 0.99
LEARNING_RATE = 5e-4


In [13]:
path_pesos_ruta = "/content/drive/MyDrive/Colab Notebooks/Letra/modelo_pesos_auto.pth"
net = DQN_Linear_Model(n_features,n_actions).to(DEVICE)

In [14]:
from dqn_agent import DQNAgent
agent = DQNAgent(
    env, net, process_state, BUFFER_SIZE, BATCH_SIZE,
    LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI,
    epsilon_f=EPSILON_MIN, epsilon_anneal_time=EPSILON_TIME,
    epsilon_decay = EPSILON_DECAY, episode_block = EPISODE_BLOCK,
    save_every=SAVE_EVERY, path_pesos=path_pesos_ruta
)

In [15]:
rewards = agent.train(EPISODES, STEPS, TOTAL_STEPS, writer_name = ENV_NAME)

  0%|          | 0/500 [00:00<?, ? episodes/s]

Episode 0 - Avg. Reward over the last 110 episodes -200.0 epsilon 1.00 total steps 200
Episode 110 - Avg. Reward over the last 110 episodes -200.0 epsilon 0.73 total steps 22200
Episode 220 - Avg. Reward over the last 110 episodes -199.70909090909092 epsilon 0.54 total steps 44168
Episode 330 - Avg. Reward over the last 110 episodes -199.4090909090909 epsilon 0.40 total steps 66103
Episode 440 - Avg. Reward over the last 110 episodes -188.01818181818183 epsilon 0.30 total steps 86785
Episode 500 - Avg. Reward over the last 110 episodes -181.42727272727274 epsilon 0.26 total steps 97321


### Video DQN

In [16]:
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900),color_depth=16)
display.start()

wrapped_env = mario_utils.wrap_env(env)
agent.record_test_episode(wrapped_env)

wrapped_env.close()

### Entrenamiento DDQN

In [17]:
path_pesos_ruta_a = "/content/drive/MyDrive/Colab Notebooks/Letra/modelo_pesos_auto_A.pth"
path_pesos_ruta_b = "/content/drive/MyDrive/Colab Notebooks/Letra/modelo_pesos_auto_B.pth"

neta = DQN_Linear_Model(n_features,n_actions).to(DEVICE)
netb = DQN_Linear_Model(n_features,n_actions).to(DEVICE)

In [18]:
from double_dqn_agent import DoubleDQNAgent
agent = DoubleDQNAgent(env, neta,netb, process_state, BUFFER_SIZE, BATCH_SIZE,
                       LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI,
                       epsilon_f=EPSILON_MIN, epsilon_anneal_time=EPSILON_TIME,
                       epsilon_decay = EPSILON_DECAY, episode_block = EPISODE_BLOCK,
                       save_every=SAVE_EVERY,
                       path_pesos_a=path_pesos_ruta_a,
                       path_pesos_b=path_pesos_ruta_b)

In [19]:
rewards = agent.train(EPISODES, STEPS, TOTAL_STEPS, writer_name = ENV_NAME)

  0%|          | 0/500 [00:00<?, ? episodes/s]

Episode 0 - Avg. Reward over the last 110 episodes -200.0 epsilon 1.00 total steps 200
Episode 110 - Avg. Reward over the last 110 episodes -200.0 epsilon 0.73 total steps 22200
Episode 220 - Avg. Reward over the last 110 episodes -199.98181818181817 epsilon 0.54 total steps 44198
Episode 330 - Avg. Reward over the last 110 episodes -197.13636363636363 epsilon 0.40 total steps 65883
Episode 440 - Avg. Reward over the last 110 episodes -175.44545454545454 epsilon 0.30 total steps 85182
Episode 500 - Avg. Reward over the last 110 episodes -165.01818181818183 epsilon 0.27 total steps 94732


### Video DDQN

In [20]:
display = Display(visible=0, size=(1400, 900),color_depth=16)
display.start()

wrapped_env = mario_utils.wrap_env(env)
agent.record_test_episode(wrapped_env)

wrapped_env.close()